In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults, ARMA
from statsmodels.tsa.arima_process import ArmaProcess
from sklearn.metrics import mean_squared_error
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from timeseries_functions import index_to_datetime, plot_all_df_columns, weekly_resample, plot_series,\
plot_series_save_fig, plot_series_and_differences, run_augmented_Dickey_Fuller_test, \
plot_autocorrelation, plot_partial_autocorrelation, plot_decomposition

In [ ]:
from timeseries_functions import make_col_vector, make_design_matrix, fit_linear_trend,\
plot_trend_data, plot_linear_trend

In [ ]:
from AR_MA_functions import get_AR_model, plot_AR_model, get_AR_model_order_BIC,\
plot_BIC_AR_model, get_MA_model, plot_MA_model

In [ ]:
from ARIMA_functions import get_ARIMA_model, plot_ARIMA_model, plot_ARIMA_resids,\
get_ARIMA_forecast, plot_ARIMA_forecast_and_CI, plot_data_plus_ARIMA_predictions, \
test_rolling_ARIMA_forecast,get_predictions_df_and_plot_rolling_ARIMA_forecast

In [ ]:
from ARIMA_functions import get_ARIMAX_model, plot_ARIMAX_model,\
plot_data_plus_ARIMAX_predictions, get_ARIMAX_predictions, get_ARIMAX_training_MSE,\
get_ARIMAX_train_test_predictions, get_ARIMAX_train_test_MSE

#### import data

In [ ]:
appointments = pd.read_csv('appointments_through_04-2018.csv', index_col=0)

In [ ]:
appointments.index = pd.to_datetime(appointments.index)

In [ ]:
appointments['Hours_Spent'] = appointments['AppointmentDuration'] / 60

In [ ]:
# group by specialty
doctor = appointments[appointments['Specialty'] == 'doctor']
RN_PA = appointments[appointments['Specialty'] == 'RN/PA']
therapist = appointments[appointments['Specialty'] == 'therapist']

In [ ]:
dr_hours = doctor.groupby(doctor.index.date)['Hours_Spent'].sum()
RNPA_hours = RN_PA.groupby(RN_PA.index.date)['Hours_Spent'].sum()
therapist_hours = therapist.groupby(therapist.index.date)['Hours_Spent'].sum()

In [ ]:
num_dr = doctor.groupby([doctor.index.date])['Provider'].nunique()
num_RNPA = RN_PA.groupby([RN_PA.index.date])['Provider'].nunique()
num_therapists = therapist.groupby([therapist.index.date])['Provider'].nunique()

In [ ]:
providers = [num_dr, num_RNPA, num_therapists]
hours = [dr_hours, RNPA_hours, therapist_hours]

Name each series

In [ ]:
num_dr.name = 'Number of Doctors'
num_RNPA.name = 'Number of RN/PAs'
num_therapists.name = 'Number of Therapists'

In [ ]:
dr_hours.name = 'Doctors Hours'
RNPA_hours.name = 'RN/PAs Hours'
therapist_hours.name = 'Therapists Hours'

#### convert index to datetime and downsample each series to weekly freq

In [ ]:
for p in providers:
    p.index = pd.to_datetime(p.index)
for h in hours:
    h.index = pd.to_datetime(h.index)

In [ ]:
providers = [num_dr, num_RNPA, num_therapists]
hours = [dr_hours, RNPA_hours, therapist_hours]

In [ ]:
num_dr = weekly_resample(num_dr)
num_RNPA = weekly_resample(num_RNPA)
num_therapists = weekly_resample(num_therapists)

In [ ]:
dr_hours = weekly_resample(dr_hours)
RNPA_hours = weekly_resample(RNPA_hours)
therapist_hours = weekly_resample(therapist_hours)

#### test for stationarity in all time series

In [ ]:
providers = [num_dr, num_RNPA, num_therapists]
hours = [dr_hours, RNPA_hours, therapist_hours]

In [ ]:
# for p in providers:  
#     print(p.name)
#     run_augmented_Dickey_Fuller_test(series=p, num_diffs=2)

In [ ]:
# for h in hours:
#     print(h.name)
#     run_augmented_Dickey_Fuller_test(series=h, num_diffs=2)

#### determine number of AR and MA terms to add for each time series

In [ ]:
# Plot ACF and PACF of first difference of each series
for p in providers:
    params = {'figure.figsize': [6,4],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
    plot_partial_autocorrelation(series=p, params=params, lags=30, alpha=0.05, title='PACF {}'.format(p.name))
    plot_autocorrelation(series=p, params=params, lags=30, alpha=0.05, title='ACF {}'.format(p.name))

In [ ]:
for h in hours:
    params = {'figure.figsize': [6,4],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
    plot_partial_autocorrelation(series=h, params=params, lags=30, alpha=0.05, title='PACF {}'.format(h.name))
    plot_autocorrelation(series=h, params=params, lags=30, alpha=0.05, title='ACF {}'.format(h.name))

#### get ARIMAX model using num_provider as exogenous variable and plot

#### Doctors

In [ ]:
start_date = '2015-01-12' # b/c start data differenced away
end_date = '2018-04-30'
end_pred = '2018-07-30'

In [ ]:
dr_results, dr_summary, dr_params, dr_residuals = get_ARIMAX_model(data=dr_hours,\
                    exog_var=num_dr, order=(5, 1,0))

In [ ]:
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plot_ARIMAX_model(data=dr_hours, order=(5,1,0), exog_var=num_dr, params=params, start=start_date,\
                 end=end_pred)

In [ ]:
# getARIMAX predictions for doctors
start_date = '2015-01-12'
end_pred = '2018-07-30'
dr_forecast = dr_results.predict(start=start_date, end=end_pred, exog=num_dr)

In [ ]:
# plot_data_plus_ARIMAX_predictions(data=dr_hours, order=(5, 1,0), start=start_date,\
#     end=end_pred, exog=num_dr, typ='levels', figsize=(8,6), title='Doctors', ylabel='', xlabel='')

In [ ]:
# plot acf/pacf of Dr ARIMAX residuals
params = {'figure.figsize': [8,4],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plot_partial_autocorrelation(series=dr_residuals, params=params, lags=30, alpha=0.05, title='PACF {}'.format('Dr ARIMAX residuals'))
plot_autocorrelation(series=dr_residuals, params=params, lags=30, alpha=0.05, title='ACF {}'.format('Dr ARIMAX residuals'))

#### RN/PA

In [ ]:
start_date = '2015-01-12' # b/c start data differenced away
end_date = '2018-04-30'
end_pred = '2018-07-30'
RNPA_order = (4,1,0)

In [ ]:
RNPA_results, RNPA_summary, RNPA_params, RNPA_residuals = get_ARIMAX_model(data=dr_hours,\
                    exog_var=num_dr, order=RNPA_order)

In [ ]:
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plot_ARIMAX_model(data=RNPA_hours, order=(5, 1,0), exog_var=num_dr, params=params, start=start_date,\
                 end=end_pred)

In [ ]:
# getARIMAX predictions for doctors
RNPA_forecast = RNPA_results.predict(start=start_date, end=end_pred, exog=num_RNPA)

In [ ]:
# plot_data_plus_ARIMAX_predictions(data=RNPA_hours, order=(5, 1,0), start=start_date,\
#     end=end_pred, exog=num_RNPA, typ='levels', figsize=(8,6), title='RN/PA', ylabel='', xlabel='')

In [ ]:
# plot acf/pacf of RN/PA ARIMAX residuals
params = {'figure.figsize': [8,4],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plot_partial_autocorrelation(series=RNPA_residuals, params=params, lags=30, alpha=0.05, title='PACF {}'.format('RN/PA ARIMAX residuals'))
plot_autocorrelation(series=RNPA_residuals, params=params, lags=30, alpha=0.05, title='ACF {}'.format('RN/PA ARIMAX residuals'))

#### therapists

In [ ]:
start_date = '2015-01-12' # b/c start data differenced away
end_date = '2018-04-30'
end_pred = '2018-07-30'
therapist_order = (4,1,0)

In [ ]:
therapist_results, therapist_summary, therapist_params, therapist_residuals = get_ARIMAX_model(data=therapist_hours,\
                    exog_var=num_therapists, order=therapist_order)

#### get data and forecast, save predictions to csv

In [ ]:
ther_forecast, ther_predictions = get_ARIMAX_predictions(data=therapist_hours, order=therapist_order,\
        start=start_date, end=end_pred, exog=num_therapists, typ='levels')

In [ ]:
ther_predictions.columns = ['Number of Hours', 'Predicted Hours']

In [ ]:
ther_predictions['Hours per Provider'] = ther_predictions['Number of Hours'] / num_therapists

In [ ]:
avg_hours_per_therapist = ther_predictions['Hours per Provider'].mean()

In [ ]:
ther_predictions['Predicted # of Providers'] = np.ceil(ther_predictions['Predicted Hours'] / avg_hours_per_therapist)

In [ ]:
# ther_predictions

In [ ]:
def ARIMAX_forecast_df_to_csv(prediction_df, num_providers, file_name):
    """Save ARIMAX predictions to csv file
    prediction_df: forecast output with datetime index, raw data column, predicted hours column
    num_providers: pandas series w/ corresponding datetime index and number of providers
    filename: name for output file
    """
    prediction_df.columns = ['Number of Hours', 'Predicted Hours']
    prediction_df['Hours per Provider'] = prediction_df['Number of Hours'] / num_providers
    avg_hours_per_provider = prediction_df['Hours per Provider'].mean()
    prediction_df['Predicted # of Providers'] = np.ceil(prediction_df['Predicted Hours'] / avg_hours_per_provider)
    prediction_df.to_csv('{}_ARIMAX_predictions.csv'.format(file_name))

In [ ]:
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plot_ARIMAX_model(data=therapist_hours, order=therapist_order, exog_var=num_therapists, params=params, start=start_date,\
                 end=end_pred)

In [ ]:
# plot acf/pacf of RN/PA ARIMAX residuals
params = {'figure.figsize': [8,4],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plot_partial_autocorrelation(series=therapist_residuals, params=params, lags=30, alpha=0.05, title='PACF {}'.format('Therapist ARIMAX residuals'))
plot_autocorrelation(series=therapist_residuals, params=params, lags=30, alpha=0.05, title='ACF {}'.format('Therapist ARIMAX residuals'))

#### import hours per provider dataframes and split into training and test data

In [ ]:
dr_df = pd.read_csv('doctors_hours_per_provider.csv', index_col=0)
RNPA_df = pd.read_csv('RNPA_hours_per_provider.csv', index_col=0)
ther_df = pd.read_csv('therapists_hours_per_provider.csv', index_col=0)

In [ ]:
train_start = '2015-01-12'
train_end = '2018-02-26'
test_start = '2018-03-05'

In [ ]:
dr_train = dr_df.loc[train_start:train_end]
dr_test = dr_df.loc[test_start:]

In [ ]:
RNPA_train = RNPA_df.loc[train_start:train_end]
RNPA_test = RNPA_df.loc[test_start:]

In [ ]:
ther_train = ther_df.loc[train_start:train_end]
ther_test = ther_df.loc[test_start:]

#### get MSE (train and test data) for each category to compare with other models

In [ ]:
train_dfs = [dr_train, RNPA_train, ther_train]
test_dfs = [dr_test, RNPA_test, ther_test]
orders = [(5,1,0), (4,1,0), (4,1,0)]
endog_col = 'Hours'
exog_col = 'Number_Providers'
names = ['Doctors', 'RN/PAs', 'Therapists']
test_start='2018-03-05'
train_start = '2015-01-12'
train_end = '2018-02-26'

In [ ]:
# get train/test predictions and MSE for all 3 categories
train_mse = []
test_mse = []
category = []
# full_dfs = []
for i in range(len(train_dfs)):
    forecast, full_df = get_ARIMAX_train_test_predictions(training_data=train_dfs[i][endog_col],\
            test_data=test_dfs[i][endog_col], order=orders[i], start='2015-01-19',\
            end='2018-04-30', exog=train_dfs[i][exog_col], typ='levels')
    train_error, test_error = get_ARIMAX_train_test_MSE(full_df, data_col='data', pred_col='forecast',\
                        train_end='2018-02-26', test_start='2018-03-05', data_name=names[i])   
    train_mse.append('training set: {}'.format(train_error))
    test_mse.append('test set: {}'.format(test_error))
    category.append('{} MSE'.format(names[i]))
mse = zip(train_mse, test_mse)
print(list(zip(category,mse)))